In [ ]:
import robotic as ry
import numpy as np

from time import sleep





def komo_solver(robots, m_objects, goals):
    
    if not isinstance(robots, list):
        robots = [robots]
    if not isinstance(m_objects, list):
        m_objects = [m_objects]
    if not isinstance(goals, list):
        goals = [goals]
    
    komo = ry.KOMO(C, phases=2, slicesPerPhase=20, kOrder=2, enableCollisions=True)  
    komo.addControlObjective([], 0)
    komo.addControlObjective([], 2)
    
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)     
    
    for i in range(len(robots)):
        komo.addObjective([1, -1], ry.FS.distance, [robots[i], m_objects[i]], ry.OT.eq) 
        komo.addModeSwitch([1,-1], ry.SY.stable, [robots[i], m_objects[i]], True) 
        komo.addObjective([2] , ry.FS.positionDiff, [m_objects[i], goals[i]], ry.OT.eq) 

    ret = ry.NLP_Solver(komo.nlp(), verbose=4).solve() 
    
    return (komo, ret)

def komo_onestate_hold(C, robot, m_object):
    komo = ry.KOMO(C, phases=1, slicesPerPhase=1, kOrder=0, enableCollisions=True)
    komo.addControlObjective([], 0)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq) 
    komo.addObjective([], ry.FS.distance, [robot, m_object], ry.OT.eq)
    
    ret = ry.NLP_Solver(komo.nlp(), verbose= 4).solve()
    return (komo, ret)


def komo_onestate_move(C, robot, m_object, goal):
    komo = ry.KOMO(C, phases=1, slicesPerPhase=1, kOrder= 1, enableCollisions=True)
    komo.addModeSwitch([0,1], ry.SY.stable, [robot, m_object], True) 
    komo.addObjective([], ry.FS.distance, [robot, m_object], ry.OT.eq)
    komo.addObjective([1, 2] , ry.FS.positionDiff, [m_object, goal], ry.OT.eq) 
    
    ret = ry.NLP_Solver(komo.nlp(), verbose= 4).solve()
    return (komo, ret)

    

def rrt_solver(C, start_state, end_state, trial_count):
    rrt = ry.PathFinder()
    rrt.setProblem(C, start_state, end_state)
    
    ret = rrt.solve()
        
    for i in range(trial_count):
        if ret.feasible:
            return ret
        ret = rrt.solve() 
        print(ret.feasible) 
         
    
    return ret


def rrt_viewer(C, ret, attach_state = None, attach = False):
    
    sleep(1)
    if not ret.feasible:
        return
    
    path = ret.x
    
    initial_state = C.getFrameState()
    
    if attach:
        
        C.setJointState(attach_state)
        C.attach(ego1, obj1)
        C.attach(ego2, obj2)
    
    
    for i in path:
        C.setJointState(i)
        C.view()
        sleep(0.03)
    
    C.setFrameState(initial_state)
    
    if attach:
        C.getFrame(obj1).unLink()
        C.getFrame(obj2).unLink()
        
def path_viewer(C, path, attach_index1, attach_index2):
    
    initial_state = C.getFrameState()
    
    C.view()

    sleep(2)
    
    for i in range(0, len(path)):
        
        
        C.setJointState(path[i])
        C.view()
        sleep(0.05)
        
        if i == attach_index1:
            C.attach(ego1, obj1)
        if i == attach_index2:
            C.attach(ego2, obj2)
            
    C.setFrameState(initial_state)
    
    C.getFrame(obj1).unLink()
    C.getFrame(obj2).unLink()
        
        






def path_mapper(path, final_size):
    original_path, hold_index = path
    original_path = np.array(original_path)
    path_len = len(original_path)

    # Sample evenly spaced indices, reserving one for final state and one for hold
    steps_to_sample = final_size - 2  # 1 for hold, 1 for final
    sample_indices = np.linspace(0, path_len - 2, steps_to_sample).astype(int)
    sampled_path = original_path[sample_indices]

    # Prepare hold and final states
    hold_state = original_path[hold_index].reshape(1, -1)
    final_state = original_path[-1].reshape(1, -1)

    # Determine best insertion point for the hold state
    insert_pos = np.searchsorted(sample_indices, hold_index)

    # Insert hold state at the right spot
    with_hold = np.insert(sampled_path, insert_pos, hold_state, axis=0)

    # Append final state
    final_result = np.vstack([with_hold, final_state])
    return final_result, insert_pos



def komo_whole_planner(c, index):
    temp = ry.Config()
    temp.addConfigurationCopy(c)
    
    temp.delFrame(f"ego{3 - index}")
    temp.delFrame(f"obj{3 - index}")
    
    print("temp view ")
    temp.view()
    sleep(5)


    ego1 = f"ego{index}"
    
    first_robot_hold, ret1 = komo_onestate_hold(temp, ego1, obj1)  
    j1_hold = first_robot_hold.getPath()[0]  # birinci tobotun cargonun yanına geldiği state
    
    
    first_robot_move, ret1 = komo_onestate_move(temp, ego1, obj1, goal1)
    j1_moved = first_robot_move.getPath()[0] # birinci robotun goal yanına geldiği state
    
    
    return (j1_hold, j1_moved)




ego1 = "ego1"
ego2 = "ego2"
obj1 = "obj1"
obj2 = "obj2"
goal1 = "goal1"
goal2 = "goal2"

robots = [ego1, ego2]
m_objects = [obj1, obj2]
goals = [goal1, goal2]



C = ry.Config()


segman_maps = ["pcg-1.g", "pcg-2.g", "pcg-1-hard.g"]

C.addFile(f"real_maps/{segman_maps[1]}")
C.view()




0

In [2]:
j1_hold, j1_moved = komo_whole_planner(C, 1)
j2_hold, j2_moved = komo_whole_planner(C, 2)

inittt = np.concatenate((j1_hold[:3], j2_hold[3:]))

print("initial")
C.setJointState(inittt)
C.view()
sleep(5)


final = np.concatenate((j1_moved[:3], j2_moved[3:]))

print("final")
C.setJointState(final)
C.view()
sleep(5)


C.setJointState(inittt)
C.attach(ego1, obj1)
C.attach(ego2, obj2)


rrt_solution = rrt_solver(C, inittt, final, 100)

if rrt_solution.feasible:
    path_viewer(C, rrt_solution.x, 3, 5)


temp view 
-- WARNING:kin.cpp:getFrame:415(-1) cannot find frame named 'ego2'
-- WARNING:kin.cpp:getFrame:415(-1) cannot find frame named 'ego1'
STACK9 /bin/python3(_PyFunction_Vectorcall
STACK8 /bin/python3(_PyEval_EvalFrameDefault
STACK7 /bin/python3(+0x198a6b) [0x560ebe037a6b]
STACK6 /bin/python3(_PyObject_MakeTpCall
STACK5 /bin/python3(+0x18ae12) [0x560ebe029e12]
STACK4 /home/alperen/.local/lib/python3.10/site-packages/robotic/_robotic.so(+0x3f21e) [0x7fa7eaa2521e]
STACK3 /home/alperen/.local/lib/python3.10/site-packages/robotic/_robotic.so(+0xc5a22) [0x7fa7eaaaba22]
STACK2 symbols2feature(FeatureSymbol, rai::Array<rai::String> const&, rai::Configuration const&, rai::ArrayDouble const&, rai::ArrayDouble const&, int)
STACK1 rai::Configuration::getFrameIDs(rai::Array<rai::String> const&) const
STACK0 rai::LogToken::~LogToken()


== ERROR:kin.cpp:getFrameIDs:446(-2) frame name 'ego1' doesn't exist


RuntimeError: kin.cpp:getFrameIDs:446(-2) frame name 'ego1' doesn't exist